# Baseline model

In [1]:
import numpy as np
import pandas as pd

### Opening data sets:

In [2]:
recipes = pd.read_csv("../raw_data/recipes.csv")

In [3]:
recipes.head()

,id,name,ingredients,ingredient_ids
0,137739,arriba baked winter squash mexican style,"['winter squash', 'mexican seasoning', 'mixed ...","[7933, 4694, 4795, 3723, 840, 5006, 6270]"
1,31490,a bit different breakfast pizza,"['prepared pizza crust', 'sausage patty', 'egg...","[5481, 6324, 2499, 4717, 6276, 1170]"
2,59389,alouette potatoes,"['spreadable cheese with garlic and herbs', 'n...","[1170, 4918, 6426, 5185, 7099, 5006, 6009, 627..."
3,5289,apple a day milk shake,"['milk', 'vanilla ice cream', 'frozen apple ju...","[4717, 7474, 2946, 150]"
4,25274,aww marinated olives,"['fennel seeds', 'green olives', 'ripe olives'...","[2587, 3437, 5002, 3184, 5324, 5068, 5058, 131..."


In [4]:
# has all ingredients, incluing the repeated ones
ingredients = pd.read_csv("../raw_data/ingredient_keys.csv")[['id','replaced','count']].drop_duplicates()

In [5]:
ingredients.head()

,id,replaced,count
0,4308,lettuce,4507
42,2744,french vanilla pudding and pie filling mix,5
43,6843,stove top stuffing mix,114
45,1910,cream cheese,9915
86,1168,cheddar,13304


In [48]:
ingredients.sort_values(by='id')

,id,replaced,count
3483,0,'s baking chocolate,2
3151,1,'s chocolate chip,13
4941,2,'s hugs chocolate,7
6931,3,'s sauce,6
5290,4,'s sour cream,2
...,...,...,...
9382,8018,ziploc bag,32
9638,8019,ziti pastum,200
7172,8020,ziti rigatus,2
2114,8021,zucchini,3948


# Buildind the base df "ingredient_pairs"

In [6]:
# ingredient_pairs = pd.DataFrame(columns = list(ingredients['id'].sort_values()),index=list(ingredients['id'].sort_values()), data=0)

In [7]:
# ingredient_pairs

# Creating loop to fill df with pair combinations

In [8]:
import ast
import time

In [9]:
### descomentar se quiser rodar a célula de baixo
# recipes_list = recipes['ingredient_ids'].to_numpy()
# recipes_list = np.array([ast.literal_eval(item) for item in recipes_list])

In [10]:
#### descomentar a de cima e essa se quiser executar de novo a criação da combinação dos ingredientes
# %%time
# for ingr_id in range(8023):
#     print(f'looking for recipes containing ingredient {ingr_id}', end='\r')
#     for idx, recipe_ingredients in enumerate(recipes_list):
#         if ingr_id in recipe_ingredients:
#             ingredient_pairs.loc[ingr_id,recipe_ingredients] += 1
#             recipe_ingredients.remove(ingr_id)
#             if not recipe_ingredients:
#                 np.delete(recipes_list, idx, 0)

In [11]:
#### descomentar se quiser criar o csv
# ingredient_pairs.to_csv('../raw_data/ingredient_pairs.csv', index=False)

In [12]:
ingredient_pairs = pd.read_csv('../raw_data/ingredient_pairs.csv')
ingredient_pairs

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,13,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,29,0,0,1,0
8019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,187,0,8,0
8020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8021,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3661,0


# Mirror the matrix

In [49]:
# mirrorring loop: gets the vertical component of the item, horizontal component, 
# and attibutes it to the rows in the ingredient_pairs df
for idx in range(8023):
    vertical = ingredient_pairs.iloc[0:int(idx+1),int(idx):int(idx+1)].T.values.tolist()[0]
    horizontal = ingredient_pairs.iloc[idx:idx+1,idx+1:].values.tolist()[0]
    ingredient_pairs.iloc[idx:(idx+1),:] = pd.DataFrame(vertical + horizontal).T

In [ ]:
ingredient_pairs

# Dividing the pairs array by the single ingredient vector

In [52]:
# vector with number of recipes for each ingredient
vector = np.diagonal(ingredient_pairs)

In [55]:
ingredient_pairs_np = np.array(ingredient_pairs)

In [56]:
combination_percentage = ingredient_pairs_np / vector

<ipython-input-56-68f2f18b1217>:1: RuntimeWarning: invalid value encountered in true_divide
  combination_percentage = ingredient_pairs_np / vector


In [57]:
combination_percentage = pd.DataFrame(combination_percentage)

In [58]:
combination_percentage

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
1,0.0,1.000000,0.166667,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
2,0.0,0.076923,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
3,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
4,0.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.000273,0.0
8019,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.002185,0.0
8020,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.0,0.000000,0.0
8021,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,...,0.0,0.0,0.026316,0.0,0.0,0.034483,0.042781,0.0,1.000000,0.0


In [59]:
#### descomentar se quiser criar o csv
# combination_percentage.to_csv('../raw_data/combination_percentage.csv', index=False)

# Code to send to front-end

In [60]:
test = combination_percentage

In [109]:
test.iloc[0:1,7000:].T.sort_values(by=0, ascending=False)

,0
7790,0.000399
7449,0.000094
7000,0.000000
7680,0.000000
7685,0.000000
...,...
7347,0.000000
8022,0.000000
7403,NaN
7664,NaN
